In [1]:
import sys, re, math, time
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
from os import path
import os,binascii
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
%load_ext autoreload
%autoreload 2
from run_experiments import *
from datahelper import *
import keras.metrics
keras.metrics.cindex_score = cindex_score
from sklearn.metrics import mean_squared_error, f1_score
from keras.models import load_model
from sklearn.model_selection import ParameterGrid
%matplotlib inline
from argparse import Namespace

Using TensorFlow backend.


In [2]:
FLAGS = argparser()
FLAGS.log_dir = FLAGS.log_dir + str(time.time()) + "/"
FLAGS.num_windows = [32, 64, 128]
FLAGS.seq_window_lengths = [4, 6]
FLAGS.smi_window_lengths = [2, 4]
FLAGS.batch_size = 256
FLAGS.num_epoch = 100
FLAGS.max_seq_len = [300, 500, 1000]
FLAGS.max_smi_len = 95
FLAGS.dataset_path= '../data/davis/'
FLAGS.problem_type = 1
FLAGS.bns = [False]
FLAGS.dropouts = [0.3, 0.5]
FLAGS.charseqset_size = CHARPROTLEN 
FLAGS.charsmiset_size = CHARISOSMILEN 
dtc_data_file = '../../data/dtc_for_deepDTA.csv'

FLAGS_dict = vars(FLAGS)

for i, val in FLAGS_dict.items():
    if type(FLAGS_dict[i]) != list:
        FLAGS_dict[i] = [val]

In [3]:
#proteins = json.load(open(FLAGS.dataset_path+"proteins.txt"), object_pairs_hook=OrderedDict)
#proteins = [i[1] for i in proteins.items()]
#dtc_train = pd.read_csv(dtc_data_file)
#with open('proteins.csv', 'w') as handler:
#    for i, seq in enumerate(list(dtc_train['fasta'].unique()) + proteins):
#        handler.write('>id'+str(i)+'\n')
#        handler.write(seq+'\n')

In [ ]:
n_repeats = 1
param_grid = ParameterGrid(FLAGS_dict)

if path.exists(FLAGS.results_pickle[0]):
    with open(FLAGS.results_pickle[0], 'rb') as handle:
        results = pickle.load(handle)
else:
    results = {}

for ind in range(len(param_grid)):
       
    params = param_grid[ind]
    cur_FLAGS = Namespace(**params)
    all_train_drugs, all_train_prots, all_train_Y = load_data(cur_FLAGS)
    val_inds = get_n_fold_by_drugs(all_train_drugs, 10)
    param_name = str(binascii.b2a_hex(os.urandom(15))).replace("'", '')

    early_stopping_callback = CustomStopper(monitor='val_loss', patience=30, start_epoch=50)
    fold_id, avg_loss, avg_cindex, avg_f1, avg_rmse = 0, 0.0, 0.0, 0.0, 0.0
    for tr_fold, val_fold in val_inds.split():
        if fold_id >= n_repeats:
            break

        model_name='checkpoints/davis_dtc_dta_'+param_name+'fold'+str(fold_id)

        checkpoint_callback = ModelCheckpoint(model_name+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        
        XD_train, XT_train, Y_train = all_train_drugs[tr_fold], all_train_prots[tr_fold], all_train_Y[tr_fold]
        XD_val, XT_val, Y_val = all_train_drugs[val_fold], all_train_prots[val_fold], all_train_Y[val_fold]

        gridmodel = build_combined_categorical(cur_FLAGS, cur_FLAGS.num_windows, cur_FLAGS.smi_window_lengths, 
                                       cur_FLAGS.seq_window_lengths, cur_FLAGS.dropouts, apply_bn=cur_FLAGS.bns)

        gridres = gridmodel.fit(([XD_train, XT_train ]), Y_train, batch_size=cur_FLAGS.batch_size, epochs=cur_FLAGS.num_epoch, 
                validation_data=( ([np.array(XD_val), np.array(XT_val) ]), np.array(Y_val))
                   ,callbacks=[early_stopping_callback, checkpoint_callback], verbose=2)
        
        gridmodel = load_model(model_name+'.h5')
        predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val)])[:, 0]
        loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)
        avg_loss += loss/n_repeats
        avg_cindex += rperf2/n_repeats
        avg_f1 += f1_score(Y_val>7, predicted_labels>7)/n_repeats
        avg_rmse += np.sqrt(mean_squared_error(Y_val, predicted_labels))/n_repeats
        fold_id += 1
    
    results[param_name] = {'FLAGS':cur_FLAGS, 'params': params, 'loss': avg_loss, 'cindex': avg_cindex,
                       'rmse':  avg_rmse, 'f1': avg_f1, 'n_repeats': n_repeats}
    with open(FLAGS.results_pickle[0], 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


Reading ../data/davis/ start
Read ../data/davis/ start
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 41811 samples, validate on 4090 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.52315, saving model to checkpoints/davis_dtc_dta_b95d022e051c4a41b3385e8fb89b97dfold0.h5
 - 5s - loss: 3.0492 - cindex_score: 0.5640 - val_loss: 0.5232 - val_cindex_score: 0.6323
Epoch 2/100
Epoch 00002: val_loss improved from 0.52315 to 0.50245, saving model to checkpoints/davis_dtc_dta_b95d022e051c4a41b3385e8fb89b97dfold0.h5
 - 3s - loss: 0.6531 - cindex_score: 0.6366 - val_loss: 0.5025 - val_cindex_score: 0.6474
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.6052 - cindex_score: 0.6742 - val_loss: 0.5241 - val_cindex_score: 0.6455
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.5878 - cindex_score: 0.6906 - val_loss: 0.5163 - val_cindex_score: 0

Epoch 52/100
Epoch 00052: val_loss did not improve
 - 3s - loss: 0.2722 - cindex_score: 0.8519 - val_loss: 0.7981 - val_cindex_score: 0.6348
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 3s - loss: 0.2706 - cindex_score: 0.8516 - val_loss: 0.8628 - val_cindex_score: 0.6297


/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Reading ../data/davis/ start
Read ../data/davis/ start
Train on 41811 samples, validate on 4090 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.48544, saving model to checkpoints/davis_dtc_dta_be10147ef965cfa3144a6b717d29272fold0.h5
 - 4s - loss: 2.9793 - cindex_score: 0.5747 - val_loss: 0.4854 - val_cindex_score: 0.6336
Epoch 2/100
Epoch 00002: val_loss improved from 0.48544 to 0.45840, saving model to checkpoints/davis_dtc_dta_be10147ef965cfa3144a6b717d29272fold0.h5
 - 3s - loss: 0.6331 - cindex_score: 0.6564 - val_loss: 0.4584 - val_cindex_score: 0.6586
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.5843 - cindex_score: 0.6935 - val_loss: 0.4720 - val_cindex_score: 0.6620
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.5683 - cindex_score: 0.7089 - val_loss: 0.4715 - val_cindex_score: 0.6779
Epoch 5/100
Epoch 00005: val_loss improved from 0.45840 to 0.44246, saving model to checkpoints/davis_dtc_dta_be10147ef965cfa3144a6b717d2927

/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


...

Epoch 53/100
Epoch 00053: val_loss did not improve
 - 8s - loss: 0.2487 - cindex_score: 0.8611 - val_loss: 0.6614 - val_cindex_score: 0.6799
Reading ../data/davis/ start
Read ../data/davis/ start
Train on 41811 samples, validate on 4090 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.51547, saving model to checkpoints/davis_dtc_dta_bea4340d11ad58c87e2fd3025fa1a23fold0.h5
 - 28s - loss: 2.9474 - cindex_score: 0.5868 - val_loss: 0.5155 - val_cindex_score: 0.5952
Epoch 2/100
Epoch 00002: val_loss improved from 0.51547 to 0.47643, saving model to checkpoints/davis_dtc_dta_bea4340d11ad58c87e2fd3025fa1a23fold0.h5
 - 9s - loss: 0.6402 - cindex_score: 0.6575 - val_loss: 0.4764 - val_cindex_score: 0.6228
Epoch 3/100
Epoch 00003: val_loss improved from 0.47643 to 0.47286, saving model to checkpoints/davis_dtc_dta_bea4340d11ad58c87e2fd3025fa1a23fold0.h5
 - 8s - loss: 0.5928 - cindex_score: 0.6910 - val_loss: 0.4729 - val_cindex_score: 0.6272
Epoch 4/100
Epoch 00004: val_loss impr

Epoch 55/100
Epoch 00055: val_loss did not improve
 - 9s - loss: 0.2338 - cindex_score: 0.8695 - val_loss: 0.7480 - val_cindex_score: 0.6260
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 9s - loss: 0.2295 - cindex_score: 0.8696 - val_loss: 0.8415 - val_cindex_score: 0.6197


/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Reading ../data/davis/ start
Read ../data/davis/ start
Train on 41811 samples, validate on 4090 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.52488, saving model to checkpoints/davis_dtc_dta_bd72a25a059237761689d87461fe97dfold0.h5
 - 30s - loss: 2.9920 - cindex_score: 0.5785 - val_loss: 0.5249 - val_cindex_score: 0.6268
Epoch 2/100
Epoch 00002: val_loss improved from 0.52488 to 0.50401, saving model to checkpoints/davis_dtc_dta_bd72a25a059237761689d87461fe97dfold0.h5
 - 9s - loss: 0.6468 - cindex_score: 0.6472 - val_loss: 0.5040 - val_cindex_score: 0.6511
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 9s - loss: 0.6014 - cindex_score: 0.6792 - val_loss: 0.5179 - val_cindex_score: 0.6525
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 9s - loss: 0.5851 - cindex_score: 0.6923 - val_loss: 0.5096 - val_cindex_score: 0.6685
Epoch 5/100
Epoch 00005: val_loss improved from 0.50401 to 0.49728, saving model to checkpoints/davis_dtc_dta_bd72a25a059237761689d87461fe9

Epoch 54/100
Epoch 00054: val_loss did not improve
 - 9s - loss: 0.2309 - cindex_score: 0.8665 - val_loss: 0.9809 - val_cindex_score: 0.6041
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 9s - loss: 0.2296 - cindex_score: 0.8665 - val_loss: 0.8858 - val_cindex_score: 0.6087
Epoch 56/100


In [ ]:
gridmodel = load_model(model_name+'.h5')

predicted_labels = gridmodel.predict([np.array(XD_train), np.array(XT_train)])[:, 0]
loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)
print(loss, rperf2)
print(np.sum(predicted_labels>7), np.sum(Y_val>7))

In [ ]:
#gridmodel = load_model(model_name+'.h5')
data_path = '../data/'
XD_test, XT_test = get_DTC_train(data_path+'dtc_test_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len, with_label=False)
XD_test, XT_test = np.asarray(XD_test), np.asarray(XT_test)
predicted_labels = gridmodel.predict([np.array(XD_test), np.array(XT_test) ])


In [ ]:
submission_data = pd.read_csv(data_path+'round_1_template.csv')
submission_data.loc[:, 'pKd_[M]_pred'] = predicted_labels
submission_data.to_csv(data_path+'submission_file.csv', index=None)